# LIME Local Explanation

In [ ]:
import os
import warnings
from pathlib import Path

import shap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

from src.utils.models_pkl import load_pickle
from src.modeling.create_data_split import split_data
from src.explanation.local.utils import plot_bar, plot_scatter
from src.explanation.local.lime_explainer import LimeExplainer
from config.config_modeling import CAT_COLS, TRAIN_SIZE, TEST_FROM_VAL, RANDOM_STATE

## Notebook settings

In [ ]:
warnings.filterwarnings("ignore")

## Define constants

In [ ]:
# Paths.
IN_PATH = Path(os.path.join("..", "..", "data", "processed")) / "data_plain.csv"
MODEL_PATH = Path(os.path.join("..", "..", "models")) / "XGB.pkl"

## Load data

In [ ]:
data = split_data(
    cols=CAT_COLS,
    df=pd.read_csv(IN_PATH),
    train_size=TRAIN_SIZE,
    test_size=TEST_FROM_VAL,
    random_state=RANDOM_STATE,
)

X_train, Y_train = data["train"]
X_train.reset_index(drop=True, inplace=True)
Y_train.reset_index(drop=True, inplace=True)

X_test, Y_test = data["test"]
X_test.reset_index(drop=True, inplace=True)
Y_test.reset_index(drop=True, inplace=True)

## Load model

In [ ]:
model = load_pickle(MODEL_PATH)

# Validate model and data.
print(roc_auc_score(Y_train, model.predict_proba(X_train)[:, 1]))
print(roc_auc_score(Y_test, model.predict_proba(X_test)[:, 1]))

## LIME Local Explanation

### Define SHAP Explainer

In [ ]:
lime_explainer = LimeExplainer(prediction_function=model.predict_proba, dataset=X_train)

### Global explanation

In [ ]:
res = lime_explainer.get_global_explanation(X_test[:1000], normalize=True)

In [ ]:
plt.figure(figsize=(8, 8))
plot_bar(dict(sorted(res.items(), key=lambda x: -x[1])), orient="h")